### 数据加载与批处理

#### 1.序列化与内存映射
在进入模型之前，原始文本已经通过Tokenizer变成了整数序列  

为了高效训练，我们不仅预测最后一个词，而是让模型在每一个位置都预测它的下一个词  

输入(x)：从i开始，长度为L。[x_i, x_{i+1}, ..., x_{i+L-1}]  
输出(y)：从i+1开始，长度为L。[y_i, y_{i+1}, ..., y_{i+L-1}]  

由于数据巨大，因此应该配合mmap使用

- 确定合法边界：max_idx = n - max_seq_length-1
- 随机采样起始点： ix = torch.randint(0, max_idx+1,(batch_size))
- 内存切片与堆叠：
  - x_stack = [dataset[i:i+max_seq_length] for i in ix]
  - y_stack = [dataset[i+1:i+max_seq_length+1] for i in ix]
- 张量化与类型转换：x = troch.from_numpy(np.array(x_stack)).to(device).long()

#### 2.高级批处理技巧：微批次 (Micro-batches) 与 动态批处理
在更复杂的场景（如分布式训练或推理）下，批处理会有更高级的变体  

• 微批次 (Micro-batches)： 在流水线并行（Pipeline Parallelism）中，我们会把一个大 Batch 进一步拆成多个更小的 Micro-batch。这样做是为了让不同的 GPU 能够像“接力赛”一样重叠计算和通信，减少等待的“气泡”时间  

• 选择性批处理 (Selective Batching)： 在推理服务中，用户的请求长度不一且到达时间随机。我们不能死等所有人凑齐，而是采用“连续批处理”技术，每生成一个 token 就检查是否有新请求进来，并动态地把它们“缝合”进当前的计算流程